# Image Captioning
<img src="static/sheep.png" align="center"/>

Image captioning is a process of **describing content in the image**. As we already know, neural networks can understand the content of the image and they also can generate text. MIX and we get NN for image captioning. As a reference paper we'll use [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/abs/1502.03044). It's a really good idea to read the paper before doing this homework.

Here's a brief overview of the paper (reference to [this repository](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning#concepts)):

## Encoder

The encoder encodes the input image with 3 color channels into a feature map which is a summary representation of all that's useful in the image. We use fully-conv architecture for it. Don't train encoder from scratch, but use pretrained CNN. E.g. you can use ResNet-101 pretrained on ImageNet from [torchvision models](https://pytorch.org/docs/stable/torchvision/models.html).

To keep encoder's output feature maps of fixed size you can use [nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d).


<img src="static/encoder.png" align="center"/>

## Decoder

The decoder's job is to look at the encoded image and generate caption **word-by-word** (note, at the `08 seminar` generated sequences token-by-token). Since we generate sequence, we'll Recurrent Neural Network (LSTM). To feed image information to RNN we 1) average `(h, w)` dimensions of image feature map to get 1D vector, then 2) transform this vector with linear layer to fit the size of the RNN's hidden state and 3) pass it as an initial hidden state $h_0$.

<img src="static/decoder_no_att.png" align="center"/>

## Attention

In a setting with attention, we want the decoder to be able to look at different parts of the image at different timestamps in the sequence. For example, while generating the word `football` it's a good idea to look at image area, where football is located!

Instead of the simple averaging (like before), we use the **weighted averaging** across all pixels. This weighted representation of the image can be concatenated with the previously generated word at each timestamp to generate the next word.

<img src="static/decoder_att.png" align="center"/>


The attention network **computes these weights**. How would you estimate the importance of a certain part of an image? You would need to be aware of the sequence you have generated so far, so you can look at the image and decide what needs describing next. For example, after you mention a man, it is logical to declare that he is holding a football.

This is exactly what the attention mechanism does – it considers the sequence generated thus far, and attends to the part of the image that needs describing next.


<img src="static/att.png" align="center"/>

We'll use **soft** attention, where the weights of the pixels add up to 1. If there are $P$ pixels in our encoded image, then at each timestep $t$:



<img src="static/weights.png" align="center"/>

## Beam search

We use a linear layer to transform the decoder's output into a score for each word in the vocabulary.

The straightforward and greedy (just like we did at the `08 seminar`) option would be to choose the word with the highest score and use it to predict the next word. But this is not optimal because the rest of the sequence hinges on that first word you choose. If that choice isn't the best, everything that follows is sub-optimal. And it's not just the first word – each word in the sequence has consequences for the ones that succeed it.

It might happen that if you'd chosen the third best word at that first step, and the second best word at the second step, and so on... that would be the best sequence you could generate.

It would be best if we could somehow not decide until we've finished decoding completely, and choose the sequence that has the highest overall score from a basket of candidate sequences.

Beam Search does exactly this.

1. At the first decode step, consider the top k candidates.
2. Generate k second words for each of these k first words.
3. Choose the top k [first word, second word] combinations considering additive scores.
4. For each of these k second words, choose k third words, choose the top k [first word, second word, third word] combinations.
5. Repeat at each decode step.

After k sequences terminate, choose the sequence with the best overall score.


<img src="static/beam_search.png" align="center"/>

## Data
We'll use [Flickr8K dataset](http://nlp.cs.illinois.edu/HockenmaierGroup/8k-pictures.html). The Flickr 8K dataset includes images obtained from the Flickr website. Each image has several captions annotated by humans. To download the dataset follow [this link](http://nlp.cs.illinois.edu/HockenmaierGroup/8k-pictures.html) (use your academic e-mail for faster approvement).

## Game rules:
Maximum score you can get for this task is **10.0**.

1. **Baseline (5 points):** implement Encoder-Decoder network which generates worthy captions (without attention and beam search).
2. **+ attention (3 points):** add attention mechanism to the implemented baseline (+ visualize attention masks).
3. **+ beam search (2 points):** implement beam search generation algorithm.

We won't use any quantitative metric to measure quality. We'll just look at your generated captions for **20 images** from Flickr8K not used during training and **5 random images** from the Internet (you can choose any). 

So the result of your homework should be:
1. Code
2. 20 + 5 images with generated captions (+ attention masks if you implemented attention mechanism)
3. Short report about things you tried and what parts you managed to implement

In [ ]:
## A LOT OF YOUR CODE HERE